<a href="https://colab.research.google.com/github/0x-s13i/deep-neural-networks/blob/main/Deep_Neural_Networks_VGG_16_on_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import sys
# print(sys.version)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np
import random as random

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyperparameters
num_epochs = 100
learning_rate = 0.001
batch_size = 128

In [5]:
# Define a transform that applies data augmentation techniques
transform = transforms.Compose([
    transforms.RandomCrop(36, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [6]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data',
                                             train=True, 
                                             transform=transform,
                                             download=True)

# Split the training dataset into train and validation datasets
train_ratio = 0.9
val_ratio = 0.1
train_size = int(train_ratio * len(train_dataset))
val_size = int(val_ratio * len(train_dataset))
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

test_dataset = torchvision.datasets.CIFAR10(root='../../data',
                                            train=False, 
                                            transform=transform)

# Create DataLoaders for the datasets
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data


In [7]:
# Setting fixed seed for random generator so that results are reproducible
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)


class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        # Initialize the base model
        super(VGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(2)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(2)
        self.conv8 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.pool4 = nn.MaxPool2d(2)
        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv13 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.pool5 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(512 * 1 * 1, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        # self.softmax = nn.Softmax(-1)

    def forward(self, x):
        # Perform forward pass
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool1(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.pool2(x)
        
        x = self.conv5(x)
        x = self.relu(x)
        x = self.conv6(x)
        x = self.relu(x)
        x = self.conv7(x)
        x = self.relu(x)
        x = self.pool3(x)
        
        x = self.conv8(x)
        x = self.relu(x)
        x = self.conv9(x)
        x = self.relu(x)
        x = self.conv10(x)
        x = self.relu(x)
        x = self.pool4(x)
        
        x = self.conv11(x)
        x = self.relu(x)
        x = self.conv12(x)
        x = self.relu(x)
        x = self.conv13(x)
        x = self.relu(x)
        x = self.pool5(x)
        
        # Flatten the output of the convolutional layers        
        x = x.view(-1, 512 * 1 * 1)

        # Pass the flattened tensor through the fully-connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        # x = self.softmax(x)
        
        return x

In [8]:
# Instantiate the model
model = VGG16()

In [9]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=5e-4*batch_size, momentum=0.9, nesterov=True)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
model = model.to(device)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        # Move the input tensors, model, and optimizer to the GPU
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

Epoch [1/100], Step [100/351], Loss: 2.0886
Epoch [1/100], Step [200/351], Loss: 1.9545
Epoch [1/100], Step [300/351], Loss: 1.8097
Epoch [2/100], Step [100/351], Loss: 1.8625
Epoch [2/100], Step [200/351], Loss: 1.8232
Epoch [2/100], Step [300/351], Loss: 1.6391
Epoch [3/100], Step [100/351], Loss: 1.8292
Epoch [3/100], Step [200/351], Loss: 1.4475
Epoch [3/100], Step [300/351], Loss: 1.5224
Epoch [4/100], Step [100/351], Loss: 1.4966
Epoch [4/100], Step [200/351], Loss: 1.3702
Epoch [4/100], Step [300/351], Loss: 1.3174
Epoch [5/100], Step [100/351], Loss: 1.1524
Epoch [5/100], Step [200/351], Loss: 0.9947
Epoch [5/100], Step [300/351], Loss: 1.2430
Epoch [6/100], Step [100/351], Loss: 0.9412
Epoch [6/100], Step [200/351], Loss: 1.2008
Epoch [6/100], Step [300/351], Loss: 1.0521
Epoch [7/100], Step [100/351], Loss: 1.0012
Epoch [7/100], Step [200/351], Loss: 1.1614
Epoch [7/100], Step [300/351], Loss: 0.8823
Epoch [8/100], Step [100/351], Loss: 0.9361
Epoch [8/100], Step [200/351], L

In [11]:
# Test the model on the validation dataset
model = model.to(device)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_dataloader:
        # Move the input tensors, model, and optimizer to the GPU
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Validation accuracy: {:.4f}'.format(accuracy))

Validation accuracy: 0.8636
